Based on the inferences from the previous part, the data set will be taken without two columns, namely, 'Time' and 'Amount'. Feature creation is hard as the nature and interaction of attributes is not given and hence that step will be passed at this time. In this notebook, a baseline model will be done after test/train split.
(Reference: https://machinelearningmastery.com/imbalanced-classification-with-the-fraudulent-credit-card-transactions-dataset/) 

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

In [2]:
df = pd.read_csv('creditcard 2.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
mydf = df.drop(['Time'], axis = 1)
myccdf = mydf.drop('Amount', axis = 1)
y = myccdf['Class']
myccdf = myccdf.drop('Class', axis = 1)
X = myccdf
y.head()


0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [5]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153


X and y are used now for the rest of the work.

In [6]:
#test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression(solver='liblinear'
                            , C=10.0, multi_class='ovr',
                           random_state=0)
logreg.fit(X_train, y_train)

LogisticRegression(C=10.0, multi_class='ovr', random_state=0,
                   solver='liblinear')

In [7]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))


Accuracy of logistic regression classifier on test set: 1.00


In [8]:
p_pred = logreg.predict_proba(X_test)
# keep probabilities for the positive outcome only
class1_probs = p_pred[:, 1]
y_pred = logreg.predict(X_test)
score_ = logreg.score(X_test, y_test)
conf_m_nr = confusion_matrix(y_test, y_pred)
report_nr = classification_report(y_test, y_pred)
precision, recall, _ = precision_recall_curve(y_test, class1_probs)
f1_nr, auc_nr = f1_score(y_test, y_pred), auc(recall, precision)
roc_auc_nr = roc_auc_score(y_test, class1_probs)
print(conf_m_nr)
print(report_nr)
print(roc_auc_nr)
print("AUC for Logistic Regression: ",auc_nr)

[[85285    11]
 [   56    91]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.89      0.62      0.73       147

    accuracy                           1.00     85443
   macro avg       0.95      0.81      0.87     85443
weighted avg       1.00      1.00      1.00     85443

0.9681099320238318
AUC for Logistic Regression:  0.7677551007740622


while F1 score is 1.00 and accuracy is also 1, AUC (Area under the Curve) is th ebest estimate for an imbalanced dataset. Taking the probabilities for the minority class ( class = 1) , AUC is .7678. It suggests for improvement by leveraging other models like Random Forest andn neural networks and sampling techniques like SMOTE, and acronym for Synthetic Minority Oversampling Technique. It is part of imblearn library. 

In [9]:
#Importing SMOTE for minority oversampling. 
#Oversamppling is olny applied to training data

from imblearn.over_sampling import SMOTE
#Oversampling the data
smote = SMOTE(random_state = 101)
X_oversample, y_oversample = smote.fit_resample(X_train, y_train)


In [10]:
classifier_o = LogisticRegression()
classifier_o.fit(X_oversample, y_oversample)
print(classification_report(y_test, classifier_o.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     85296
           1       0.06      0.92      0.11       147

    accuracy                           0.98     85443
   macro avg       0.53      0.95      0.55     85443
weighted avg       1.00      0.98      0.99     85443



In [11]:
y_pred = classifier_o.predict(X_test)
print('Accuracy of logistic regression classifier on test set with oversampled training: {:.2f}'.format(logreg.score(X_test, y_test)))


Accuracy of logistic regression classifier on test set with oversampled training: 1.00


In [12]:
p_pred =  classifier_o.predict_proba(X_test)
# keep probabilities for the positive outcome only
class1_probs = p_pred[:, 1]
y_pred =  classifier_o.predict(X_test)
score_ =  classifier_o.score(X_test, y_test)
conf_m_o = confusion_matrix(y_test, y_pred)
report_o = classification_report(y_test, y_pred)
precision_o, recall_o, _ = precision_recall_curve(y_test, class1_probs)
f1_o = f1_score(y_test, y_pred)
auc_o =  auc(recall_o, precision_o)
roc_auc_o = roc_auc_score(y_test, class1_probs)

In [13]:
print(conf_m_o)

[[83201  2095]
 [   12   135]]


In [14]:
print("AUC for Logistic Regression with oversampoling using SMOTE: ",auc_o)

AUC for Logistic Regression with oversampoling using SMOTE:  0.767793241256793


In [15]:
print(roc_auc_o)

0.9829752126887146


WIth th emetric being AUC as the chosen metric for imbalanced data sets, there is only a very small improvement with SMOTE over normal random sampling with logistic regression as modelling methoid.
0.7677551007740622 vs 0.767793241256793

Trying out RandomForest with normal sampling to findn the most useful features.

In [16]:
from sklearn.ensemble import RandomForestClassifier


In [17]:
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(X_train, y_train)
importances = rfc.feature_importances_


In [18]:
print(importances)



[0.01326846 0.00982796 0.01029522 0.02207807 0.01264187 0.01488068
 0.01716239 0.01263355 0.0151335  0.04174037 0.10428165 0.05014706
 0.01292686 0.11635846 0.01568078 0.06419459 0.27484529 0.06807631
 0.0157938  0.01500941 0.00959757 0.0077592  0.00601102 0.00913243
 0.01430109 0.02910649 0.009832   0.00728391]


In [19]:
cols = X_train.columns


In [20]:

col_importances = pd.DataFrame(importances, cols)

In [21]:
col_importances.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, V1 to V28
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       28 non-null     float64
dtypes: float64(1)
memory usage: 1.5+ KB


In [22]:
col_importances.head()

,0
V1,0.013268
V2,0.009828
V3,0.010295
V4,0.022078
V5,0.012642


In [23]:
imp_cols = col_importances[col_importances[0]>0.05]

In [24]:
imp_cols

,0
V11,0.104282
V12,0.050147
V14,0.116358
V16,0.064195
V17,0.274845
V18,0.068076


Feature selection of more important featues (importance > .05)
may help improve the model performance. With that notion, a subset of predictor variables are taken from training and test datasets. To differentiate the subset, a suffix, '-n',  is used.

In [25]:
X_train_n = X_train[['V11','V12', 'V14','V16','V17','V18']]
X_test_n = X_test[['V11','V12', 'V14','V16','V17','V18']]

In [26]:
X_train_n.head()
X_test_n.head()
y_train_n = y_train
y_test_n = y_test

Using minority oversampling with SMOTE

In [27]:
X_train_n_os, y_train_n_os = smote.fit_resample(X_train_n, y_train_n)


In [28]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)
rfc.fit(X_train_n_os, y_train_n_os)

from sklearn import metrics
pred_train_os = np.argmax(rfc.oob_decision_function_,axis=1)
print(metrics.roc_auc_score(y_train_n_os, pred_train_os))
y_pred_n = rfc.predict(X_test_n)
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
accuracy_score(y_test_n, y_pred_n)
train_probs = rfc.predict_proba(X_train_n_os)[:,1] 
probs = rfc.predict_proba(X_test_n)[:, 1]
train_predictions = rfc.predict(X_train_n_os)
print(f'Train ROC AUC Score for Oversampling minority: {roc_auc_score(y_train_n_os, train_probs)}')
print(f'Test ROC AUC  Score: {roc_auc_score(y_test_n, probs)}')
cm = confusion_matrix(y_test_n, y_pred_n)
print(cm)
precision_n_os, recall_n_os, _ = precision_recall_curve(y_test_n, probs)
f1_n_os, auc_n_os = f1_score(y_test_n, y_pred_n), auc(recall_n_os, precision_n_os)
print("AUC for oversampling:",auc_n_os)

0.9995452695471287
Train ROC AUC Score for Oversampling minority: 1.0
Test ROC AUC  Score: 0.9561945627997963
[[85239    57]
 [   29   118]]
AUC for oversampling: 0.7778003533619295


Attempting undersampling for the majority class (Class = 0) now.
(REF. : https://www.kaggle.com/residentmario/advanced-under-sampling-and-data-cleaning)

In [29]:
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import EditedNearestNeighbours


In [30]:
enn = EditedNearestNeighbours()
X_train_n_enn, y_train_n_enn = enn.fit_resample(X_train_n, y_train_n)

In [31]:
rfc_enn = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)
rfc_enn.fit(X_train_n_enn, y_train_n_enn)



RandomForestClassifier(oob_score=True, random_state=42)

In [32]:
pred_train_enn = np.argmax(rfc_enn.oob_decision_function_,axis=1)
print(metrics.roc_auc_score(y_train_n_enn, pred_train_enn))



0.8999723416559805


In [33]:
y_pred_n_enn = rfc_enn.predict(X_test_n)
accuracy_score(y_test_n, y_pred_n_enn)
train_probs_enn = rfc_enn.predict_proba(X_train_n_enn)[:,1] 
probs_enn = rfc_enn.predict_proba(X_test_n)[:, 1]
train_predictions_enn = rfc_enn.predict(X_train_n_enn)
print(f'Train ROC AUC Score for undesampling majority: {roc_auc_score(y_train_n_enn, train_probs_enn)}')
print(f'Test ROC AUC  Score : {roc_auc_score(y_test_n, probs_enn)}')
cm_enn = confusion_matrix(y_test_n, y_pred_n_enn)
print(cm_enn)
precision_n_enn, recall_n_enn, _ = precision_recall_curve(y_test_n, probs_enn)
f1_n_enn, auc_n_enn = f1_score(y_test_n, y_pred_n_enn), auc(recall_n_enn, precision_n_enn)
print("AUC for undersampling with ENN technique and RFC: ",auc_n_enn)

Train ROC AUC Score for undesampling majority: 1.0
Test ROC AUC  Score : 0.9235698382710802
[[85278    18]
 [   35   112]]
AUC for undersampling with ENN technique and RFC:  0.7789158948343924


In [34]:
undersample = TomekLinks( )
#X_tl, y_tl, id_tl = tl.fit_sample(X, y)
#undersample = NearMiss(version=1, n_neighbors=3)

In [35]:
X_train_n_us, y_train_n_us = undersample.fit_resample(X_train_n, y_train_n)

In [36]:
rfc_us = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)
rfc_us.fit(X_train_n_us, y_train_n_us)

pred_train_us = np.argmax(rfc_us.oob_decision_function_,axis=1)
print(metrics.roc_auc_score(y_train_n_us, pred_train_us))

y_pred_n_us = rfc_us.predict(X_test_n)
accuracy_score(y_test_n, y_pred_n_us)
train_probs_us = rfc_us.predict_proba(X_train_n_us)[:,1] 
probs_us = rfc_us.predict_proba(X_test_n)[:, 1]
train_predictions_us = rfc_us.predict(X_train_n_us)
print(f'Train ROC AUC Score for undesampling majority: {roc_auc_score(y_train_n_us, train_probs_us)}')
print(f'Test ROC AUC  Score : {roc_auc_score(y_test_n, probs_us)}')
cm_us = confusion_matrix(y_test_n, y_pred_n_us)
print(cm_us)
precision_n_us, recall_n_us, _ = precision_recall_curve(y_test_n, probs_us)
f1_n_us, auc_n_us = f1_score(y_test_n, y_pred_n_us), auc(recall_n_us, precision_n_us)
print("AUC for undersampling with TomekLinks and RFC: ",auc_n_us)

0.8811066551450022
Train ROC AUC Score for undesampling majority: 1.0
Test ROC AUC  Score : 0.9270555788438055
[[85288     8]
 [   40   107]]
AUC for undersampling with TomekLinks and RFC:  0.7997156183990267


Combining oversampling  and undersampling in RFC:

In [37]:
# import the SMOTEENN.
from imblearn.combine import SMOTEENN

# create the  object with the desired samplig strategy.
smoenn = SMOTEENN(sampling_strategy='minority')

# fit the object to our training data.
X_train_smtenn, y_train_smtenn = smoenn.fit_resample(X_train_n, y_train_n)


In [38]:
rfc_smtenn = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)
rfc_smtenn.fit(X_train_smtenn, y_train_smtenn)

pred_train_smtenn = np.argmax(rfc_smtenn.oob_decision_function_,axis=1)
print(metrics.roc_auc_score(y_train_smtenn, pred_train_smtenn))

y_pred_smtenn = rfc_smtenn.predict(X_test_n)
accuracy_score(y_test_n, y_pred_smtenn)
train_probs_smtenn = rfc_smtenn.predict_proba(X_train_smtenn)[:,1] 
probs_smtenn = rfc_smtenn.predict_proba(X_test_n)[:, 1]
train_predictions_smtenn = rfc_smtenn.predict(X_train_smtenn)
print(f'Train ROC AUC Score for undesampling majority: {roc_auc_score(y_train_smtenn, train_probs_smtenn)}')
print(f'Test ROC AUC  Score : {roc_auc_score(y_test_n, probs_smtenn)}')
cm_smtenn = confusion_matrix(y_test_n, y_pred_smtenn)
print(cm_smtenn)
precision_smtenn, recall_smtenn, _ = precision_recall_curve(y_test_n, probs_smtenn)
f1_smtenn, auc_smtenn = f1_score(y_test_n, y_pred_smtenn), auc(recall_smtenn, precision_smtenn)
print("AUC for sampling with SMTENN: ",auc_smtenn)

0.9998359770946955
Train ROC AUC Score for undesampling majority: 1.0
Test ROC AUC  Score : 0.9535464415554256
[[85206    90]
 [   27   120]]
AUC for sampling with SMTENN:  0.7673083800387608


So there is some improvement in AUC with undersampling and feature selection compared to ovesampling or combined oversampling and undersampling. Also Tomeklinks undersampling ( AUC = 0.7997156183990267) better in this dataset than ENN.
(Ref: https://www.kdnuggets.com/2020/01/classify-rare-event-machine-learning-algorithms.html). 
An attempt for  KNN will be tried now.

In [39]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
knn15 = KNeighborsClassifier(n_neighbors = 15)

In [41]:
# consider undersampled training set
knn15.fit(X_train_n_us,y_train_n_us)

KNeighborsClassifier(n_neighbors=15)

In [43]:
y_pred_knn15 = knn15.predict(X_test_n)
cm15 = confusion_matrix(y_test_n,y_pred_knn15)
probs_knn15 = knn15.predict_proba(X_test_n)[:, 1]
precision_knn15, recall_knn15, _ = precision_recall_curve(y_test_n, probs_knn15)
f1_knn15, auc_knn15 = f1_score(y_test_n, y_pred_knn15), auc(recall_knn15, precision_knn15)

roc_auc_acore_knn15 = roc_auc_score(y_test_n,probs_knn15)

print("KNN results for  n_neighbors = 15")
print(cm15)
print(roc_auc_acore_knn15 )
print("AUC for KNN n = 15: ",auc_knn15)

KNN results for  n_neighbors = 15
[[85283    13]
 [   45   102]]
0.9214596596470138
AUC for KNN n = 15:  0.8088955930868675


In [44]:
#import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [45]:
#In case of classifier like knn the parameter to be tuned is n_neighbors
param_grid = {'n_neighbors':np.arange(1,50)}

In [46]:
knn = KNeighborsClassifier()
knn_cv= GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(X_train_n_us,y_train_n_us)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [47]:
knn_cv.best_score_


0.9995284438647536

In [48]:
knn_cv.best_params_

{'n_neighbors': 3}

In [49]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_n_us,y_train_n_us)
y_pred_knn = knn.predict(X_test_n)
cm_knn = confusion_matrix(y_test_n,y_pred_knn)
probs_knn = knn.predict_proba(X_test_n)[:,1]
roc_auc_score_knn = roc_auc_score(y_test_n,probs_knn)
precision_knn, recall_knn, _ = precision_recall_curve(y_test_n, probs_knn)
f1_knn, auc_knn = f1_score(y_test_n, y_pred_knn), auc(recall_knn, precision_knn)

print("KNN results for  n_neighbors = 3")

print(cm_knn)
print("ROC_ AUC Score: ",roc_auc_score_knn )
print("AUC: ",auc_knn)


KNN results for  n_neighbors = 3
[[85290     6]
 [   37   110]]
ROC_ AUC Score:  0.9080612595816793
AUC:  0.8521320928776908



KNN(n = 3) with undersampling resulted in an AUC of 0.8521320928776908
and it is indeed better than Randomforest Classifier with underrsampling.

To conclude with the usual classification techniques so far, KNN performed better than Random Forest and logistic regression. Under sampling of majority class proved to be bettter than oversampling of minority class or combined over and under sampling.
The specific algorithms for imbalanced data are also worth exploring in the future iterationsn of this project.